# Python для анализа данных

## Веб-скрэйпинг: переход по ссылкам, скачивание файлов

*Автор: Татьяна Рогович, НИУ ВШЭ*

На прошлом занятии мы с вами работали с табличными данными. Второй очень частой задачей для скрэйпинга является автоматический переход по ссылкам. Обычно мы встречаемся с двумя сценариями: переход по нумерованным страницам (обычно это выдача поиска или некоторый упорядоченный архив документов) и переход по определенным ссылкам на странице. Сегодня будем учиться делать и то, и то.

Давайте начнем с учебного примера.

# Задача

По ссылке http://py4e-data.dr-chuck.net/known_by_Lilian.html список людей, которых знает Лилиан. Нужно найти ее 18-го друга (считаем с одного) и перейти по ссылке (там будет список людей, которых знает этот друг). Какой имя вы извлечете, если эту операцию повторить 7 раз? То есть нам нужно найти 18-го друга 6-го человека.

Прежде всего изучите исходный код страницы. В каком теге лежат ссылки?

In [11]:
# решение
import requests 
from bs4 import BeautifulSoup

friends = requests.get('http://py4e-data.dr-chuck.net/known_by_Lilian.html').text
soup = BeautifulSoup(friends, 'lxml')
print(soup.find_all('a')[17]) # ссылки лежат в тэге 'a', находим 18-го друга

<a href="http://py4e-data.dr-chuck.net/known_by_Tigan.html">Tigan</a>


Обратите внимание, что нужная нам информация лежит в атрибуте href, достать текст, как мы делали раньше не поможет.

In [12]:
print(soup.find_all('a')[17].get('href')) # с помощью метода get достаем информацию из атрибут href

http://py4e-data.dr-chuck.net/known_by_Tigan.html


Теперь осталось упаковать все в цикл.

In [18]:
friends = requests.get('http://py4e-data.dr-chuck.net/known_by_Lilian.html').text
soup = BeautifulSoup(friends, 'lxml')

for ix in range(6):
    link = soup.find_all('a')[17].get('href')
    print(link)
    soup = BeautifulSoup(requests.get(link).text, 'lxml')
    
print('Ответ: '+soup.find_all('a')[17].text)

http://py4e-data.dr-chuck.net/known_by_Tigan.html
http://py4e-data.dr-chuck.net/known_by_Mickey.html
http://py4e-data.dr-chuck.net/known_by_Marvin.html
http://py4e-data.dr-chuck.net/known_by_Yago.html
http://py4e-data.dr-chuck.net/known_by_Daood.html
http://py4e-data.dr-chuck.net/known_by_Jillian.html
Ответ: Bradly


## Скачивание файлов

Кстати, еще одно применение скрэйпинга, о котором мы пока не поговорили - это автоматизированное скачивание файлов. Например, вы хотите скачать научные статьи по определенному ключевому слову или прайс-листы поставщика, которые он загружает на сайт в эскеле.

Давайте посмотрим, как скачивать файлы на примере pdf и заодно попробуем походить по ссылкам. Кстати, этот процесс еще часто называется spidering или crawling, потому что ваш скрипт как паучок ползет по ссылкам (отсюда и название первых поисковых роботов - spider).

Давайте попробуем сохранить дипломные работы выпускников факультета компьютерных наук ВШЭ.

Мы уже отредактировали фильтры и ссылка их запомнила. Позже сегодня посмотрим как можно самим заполнять такие поля с помощью Selenium.

https://www.hse.ru/edu/vkr/?faculty=120026365&author=&supervisor=&year=&title=&level=0&rating=&program=g122722324%3Bg134854782%3Bp135181588%3Bp220156607%3Bp135181634%3Bg122392182%3Bp135181636&textAvailable=2&lang=en

In [21]:
link = 'https://www.hse.ru/edu/vkr/?faculty=120026365&author=&supervisor=&year=&title=&level=0&rating=&program=g122722324%3Bg134854782%3Bp135181588%3Bp220156607%3Bp135181634%3Bg122392182%3Bp135181636&textAvailable=2&lang=en'
soup = BeautifulSoup(requests.get(link).text, 'lxml')

Давайте для начала поэкспериментируем с первым студентом.

In [34]:
print(len(soup.find_all('a', {'class':'link'})))
soup.find_all('a', {'class':'link'})[:10]

108


[<a class="link" href="https://www.hse.ru/cookie.html" target="_blank" title="Пройти по ссылке">здесь</a>,
 <a class="link" href="https://www.hse.ru/data_protection_regulation" target="_blank" title="Пройти по ссылке">здесь</a>,
 <a class="link no-visited with-icon with-icon_flag-spb" href="//spb.hse.ru/">Санкт-Петербург</a>,
 <a class="link no-visited with-icon with-icon_flag-nn" href="//nnov.hse.ru/">Нижний Новгород</a>,
 <a class="link no-visited with-icon with-icon_flag-perm" href="https://perm.hse.ru/">Пермь</a>,
 <a class="link link_no-visited link_no-underline" href="/en/edu/vkr/">EN</a>,
 <a class="link link_dark no-visited" href="//www.hse.ru/search/search.html?simple=0&amp;searchid=2284688">Расширенный поиск по сайту</a>,
 <a class="link no-visited header_breadcrumb__link" href="//www.hse.ru"><span>Национальный исследовательский университет «Высшая школа экономики»</span></a>,
 <a class="link" href="/edu/vkr/296302309">Disparity Estimation via Neural Networks</a>,
 <a class="

Видим, что ссылок очень много, а нам нужны только те, которые ведут на дипломные работы. Поиграем в детектива и обнаружим, что нужные нам ссылки выглядят следующим образом: 'edu/vkr/296302309'.

In [35]:
for link in soup.find_all('a'):
    if 'edu/vkr' in link.get('href'):
        print(link)

<a class="link link_no-visited link_no-underline" href="/en/edu/vkr/">EN</a>
<a class="link" href="/edu/vkr/296302309">Disparity Estimation via Neural Networks</a>
<a class="link" href="/edu/vkr/296305514">Empirical Study of Deep Neural Network Loss Surfaces</a>
<a class="link" href="/edu/vkr/296303935">Multiple-choice Question Answering in the Russian Language</a>
<a class="link" href="/edu/vkr/296279411">Communication Complexity and its Applications</a>
<a class="link" href="/edu/vkr/296306052">Deep Learning Approaches for Classification Subcellular Protein Patterns in Human Cells</a>
<a class="link" href="/edu/vkr/296306124">Sequential Learning of Sparse ResNet Blocks</a>
<a class="link" href="/edu/vkr/296306016">Human Pose Estimation With Deep Structured Models</a>
<a class="link" href="/edu/vkr/219430191">Text Embeddings for Recommender Systems)</a>
<a class="link" href="/edu/vkr/219398831">Generators of Words Definitions</a>
<a class="link" href="/edu/vkr/219430466">Learning Spac

Ага, уже лучше, но видим, что еще остались "лишние" ссылки. Зато видим, что во всех нужных нам после слэша цифры.

In [95]:
import re

for link in soup.find_all('a'):
    if len(re.findall(r'/edu/vkr/\d+', link.get('href'))):
        vkr_link = re.findall(r'/edu/vkr/\d+', link.get('href'))[0]
        print(vkr_link)

/edu/vkr/206745575
/edu/vkr/182863926
/edu/vkr/182864188
/edu/vkr/182948750
/edu/vkr/183280945
/edu/vkr/182948863
/edu/vkr/183155936
/edu/vkr/182647987


Отлично! Теперь давайте разберемся с первым студентом, а потом упакуем все в цикл.

In [96]:
for link in soup.find_all('a'):
    if len(re.findall(r'/edu/vkr/\d+', link.get('href'))):
        vkr_link = re.findall(r'/edu/vkr/\d+', link.get('href'))[0]
        break

In [97]:
student = 'https://www.hse.ru' + vkr_link # в ссылке, которую мы скачали, не хранится домен, добавим его
student_soup = BeautifulSoup(requests.get(student).text, 'lxml')

Играем в детективов на самой странице и понимаем, что нам нужна ссылка, в которой есть текст "Текст работы".

In [98]:
for x in student_soup.find_all('a'):
    if x.text == 'Текст работы':
        print(x)
        file_link = x.get('href')
        print(file_link)

<a class="link" href="http://lms.hse.ru/ap_service.php?getwork=1&amp;guid=6288C78F-42A8-4F36-9439-D09B05EE3384">Текст работы</a>
http://lms.hse.ru/ap_service.php?getwork=1&guid=6288C78F-42A8-4F36-9439-D09B05EE3384


Вроде бы нашли. Теперь попробуем сохранить файл. Файлы студенты загружают в разных форматах, давайте скачивать только pdf. Иногда название файла будет прямо в ссылке, но это, увы, не наш случай. Будем проверять, что лежит по ссылке с помощью метода headers, который показывает, что находится в ответе по запросу.

In [103]:
course.headers['content-type']

'application/pdf'

In [108]:
course = requests.get(file_link, stream=True) # потоковое чтение файла, потому pdf может быть большим и не уместиться в памяти
if course.headers['content-type'] == 'application/pdf':
    fh = open('test.pdf', 'wb') # wb - запись байтовой информации
    fh.write(course.content) # считываем туда "содержание" файла по ссылке
    fh.close()

Осталась одна проблема, что при запуске в цикле было бы неплохо автоматически генерировать имя файла. Давайте забирать из тэга `a` название работы, когда мы находим ссылку на ВКР. Давайте соберем все вместе.

In [109]:
link = 'https://www.hse.ru/edu/vkr/?faculty=120026365&author=&supervisor=&year=&title=&level=0&rating=&program=g122722324%3Bg134854782%3Bp135181588%3Bp220156607%3Bp135181634%3Bg122392182%3Bp135181636&textAvailable=2&lang=en'
soup = BeautifulSoup(requests.get(link).text, 'lxml')

for link in soup.find_all('a'):
    if len(re.findall(r'/edu/vkr/\d+', link.get('href'))):
        vkr_link = re.findall(r'/edu/vkr/\d+', link.get('href'))[0]
        vkr_title = link.text
        
        student = 'https://www.hse.ru' + vkr_link
        student_soup = BeautifulSoup(requests.get(student).text, 'lxml')
        
        for x in student_soup.find_all('a'):
            if x.text == 'Текст работы':
                file_link = x.get('href')
                
        course = requests.get(file_link, stream=True) # потоковое чтение файла, потому pdf может быть большим и не уместиться в памяти
        if course.headers['content-type'] == 'application/pdf':
            fh = open(vkr_title+'.pdf', "wb") # создаем файл
            fh.write(course.content) # считываем туда "содержание" файла по ссылке
            fh.close()

Получилось! И последнее. У нас было 28 работ на 3 страницах, давайте по страницам тоже пройдемся. Кликните в браузере на страницу 2 по ссылке, а потом обратно на 1. Обратите внимание, что ссылка в браузере поменялась и теперь в ней появился параметр page. Скопируем новую ссылку для первой страницы.

In [110]:
page = 1 # создаю переменную и с помощью форматирования строк добавляю ее в ссылку после page=
link = f'https://www.hse.ru/edu/vkr/index.html?lang=en&page={page}&faculty=120026365&program=g122722324%3Bg134854782%3Bp135181588%3Bp220156607%3Bp135181634%3Bg122392182%3Bp135181636&textAvailable=2'

In [111]:
link # проверили, что работает

'https://www.hse.ru/edu/vkr/index.html?lang=en&page=1&faculty=120026365&program=g122722324%3Bg134854782%3Bp135181588%3Bp220156607%3Bp135181634%3Bg122392182%3Bp135181636&textAvailable=2'

In [112]:
for page in range(1,4):
    link = f'https://www.hse.ru/edu/vkr/index.html?lang=en&page={page}&faculty=120026365&program=g122722324%3Bg134854782%3Bp135181588%3Bp220156607%3Bp135181634%3Bg122392182%3Bp135181636&textAvailable=2'
    soup = BeautifulSoup(requests.get(link).text, 'lxml')

    for link in soup.find_all('a'):
        if len(re.findall(r'/edu/vkr/\d+', link.get('href'))):
            vkr_link = re.findall(r'/edu/vkr/\d+', link.get('href'))[0]
            vkr_title = link.text

            student = 'https://www.hse.ru' + vkr_link
            student_soup = BeautifulSoup(requests.get(student).text, 'lxml')

            for x in student_soup.find_all('a'):
                if x.text == 'Текст работы':
                    file_link = x.get('href')

            course = requests.get(file_link, stream=True) # потоковое чтение файла, потому pdf может быть большим и не уместиться в памяти
            if course.headers['content-type'] == 'application/pdf':
                fh = open(vkr_title+'.pdf', "wb") # создаем файл
                fh.write(course.content) # считываем туда "содержание" файла по ссылке
                fh.close()

Готово!